## Spend optimisarion across two channels using Time Delay Neural Network. 

<img src="images/Capture.png" />

We use fastai library to prepara data (embeddings, normalisations)

In [4]:
%reload_ext autoreload
%autoreload 2
from fastai.tabular import *
import os
import pandas as pd
import datetime
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(r"C:\Users\makar\fastai\interaction_interpretability\1. madex")
sys.path.append(r"C:\Users\makar\fastai\interaction_interpretability\2. glider")
sys.path.append(r"C:\Users\makar\fastai\interaction_interpretability\2. glider\models\autoint")
from sampling_and_inference import generate_perturbation_dataset_autoint
from neural_interaction_detection import detect_interactions
import os
import logging
from tqdm import tqdm
import warnings
import pickle
import numpy as np
import argparse
import torch
import torch.optim as optim
from utils.global_interaction_utils import *
import torch.multiprocessing as multiprocessing

In [5]:
from ipywidgets import widgets

uploader = widgets.FileUpload()
display(uploader)


FileUpload(value={}, description='Upload')

In [6]:
[uploaded_file] = uploader.value
print (list(uploader.value.values())[0].keys())
online=pd.read_csv(io.BytesIO(list(uploader.value.values())[0]["content"]))

dict_keys(['metadata', 'content'])


Loading Data : spend- marketing spend
Online-online sales

In [7]:
PATHD=Config().data_path()
spend=pd.read_csv(os.path.join(PATHD,r'ecommerce/Marketing_Spend.csv'))
#print(spend.columns[0])
spend.rename(columns={spend.columns[0]:'Date'},inplace=True)
spend['Date']=spend['Date'].apply(pd.to_datetime)
#spend.columns[0]='Date'
#online=pd.read_csv(os.path.join(PATHD,r'ecommerce/Online.csv'))
online['Date']=online['Date'].apply(lambda x: pd.to_datetime(x,format="%Y%m%d"))
#spend
spend_with_sales=pd.merge(online,spend,how="inner",on="Date")

Grouping sales by Date and Products code (we don't need further granularity)
Extracting date feature that might be imports: Day of week, month etc.

In [8]:
online_agg= online.groupby(['Date','Product Category (Enhanced E-commerce)'] ).agg('sum')['Revenue'].reset_index()
spend_with_sales1=pd.merge(online_agg,spend,how="inner",on="Date")
add_datepart(spend_with_sales1, "Date", drop=False)

,Date,Product Category (Enhanced E-commerce),Revenue,Offline Spend,Online Spend,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,2017-01-01,Apparel,4483.80,4500,2424.50,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
1,2017-01-01,Bags,3224.35,4500,2424.50,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
2,2017-01-01,Drinkware,5437.67,4500,2424.50,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
3,2017-01-01,Fun,992.52,4500,2424.50,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
4,2017-01-01,Headgear,1264.49,4500,2424.50,2017,1,52,1,6,1,False,True,False,True,False,True,1483228800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,2017-12-31,Lifestyle,49.97,4000,2058.75,2017,12,52,31,6,365,True,False,True,False,True,False,1514678400
3900,2017-12-31,Nest,3601.00,4000,2058.75,2017,12,52,31,6,365,True,False,True,False,True,False,1514678400
3901,2017-12-31,Nest-USA,12151.39,4000,2058.75,2017,12,52,31,6,365,True,False,True,False,True,False,1514678400
3902,2017-12-31,Office,21.00,4000,2058.75,2017,12,52,31,6,365,True,False,True,False,True,False,1514678400


Extracting features: small_cat_vars are categorical features are non-numberical in nature. Embeddings will be applied to them (using fastai functionality mechanisms); small_cont_vars -continous, applied as they are
test set is date>= November

In [9]:
small_cat_vars= ['Product Category (Enhanced E-commerce)','Dayofweek', 'Year', 'Month', 'Day','Date']
small_cont_vars  =  ['Offline Spend', 'Online Spend']
small_df = spend_with_sales1[small_cat_vars + small_cont_vars + ['Revenue']]
small_df_test=small_df[small_df['Date']>datetime.datetime(2017,11,1)]
small_df_test.drop('Revenue',inplace=True,axis=1)
small_df_train=small_df[small_df['Date']<=datetime.datetime(2017,11,1)]

C:\Users\makar\Anaconda2\envs\fastai\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Constructing databucnh: 80% of data to train, 20% to validation; No logs

In [10]:
categorify = Categorify(small_cat_vars, small_cont_vars)
categorify(small_df_train)
categorify(small_df_test, test=True)
dep_var = 'Revenue'
#doc(FloatList)
path='data/ecommerce'
small_cat_vars=['Product Category (Enhanced E-commerce)','Dayofweek', 'Year', 'Month', 'Day']
data = (TabularList.from_df(small_df_train, path=path, cat_names=small_cat_vars, cont_names=small_cont_vars, procs=[FillMissing, Categorify, Normalize],)
        .split_by_idx(range(int(len(small_df_train)*0.8),len(small_df_train)))
        .label_from_df(cols=[dep_var], label_cls=FloatList, log=True)
        .add_test(TabularList.from_df(small_df_test, path=path, cat_names=small_cat_vars, cont_names=small_cont_vars))
        .databunch())
max_log_y = np.log(np.max(small_df_train['Revenue'])*1.2)
y_range = torch.tensor([0, max_log_y], device=defaults.device)

C:\Users\makar\Anaconda2\envs\fastai\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Tabular learner with 2 hidden layers 

In [11]:
learn = tabular_learner(data, layers=[500,100], ps=[0.001,0.01], emb_drop=0.04, 
                        y_range=y_range, metrics=exp_rmspe)

Fit one cycle; learning rate was found in a separate analysis

In [12]:
learn.fit_one_cycle(5, 1e-2, wd=0.2)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,exp_rmspe,time
0,3.432148,1.719823,6.145424,01:00
1,2.300339,1.717774,3.967928,01:02
2,1.846883,1.549483,5.993158,01:15
3,1.599965,1.652824,3.338054,01:08
4,1.410046,1.642094,3.200503,01:06


epoch,train_loss,valid_loss,exp_rmspe,time
0,1.253978,1.763900,3.472735,01:03
1,1.298977,1.654479,3.896964,01:08
2,1.293217,1.691865,3.326687,01:06
3,1.257797,1.698217,4.255245,01:04
4,1.188649,1.691175,3.838983,01:02


Generating predictions for base case scenario

In [13]:
preds=learn.get_preds(ds_type=DatasetType.Test)

Interations widget

In [22]:
from ipywidgets import interact,interactive

Putting predictions for thhe test data into a dat frame

In [14]:
small_df_test_tr=spend_with_sales1[small_cat_vars + small_cont_vars + ['Revenue','Date']]
small_df_test_tr=small_df_test_tr[small_df_test_tr['Month']<=5]
#print(data.label_list.train.x.classes)
for proc in data.label_list. get_processors()[0][0].procs:
    proc.apply_test(small_df_test_tr)

databunch=TabularList.from_df(small_df_test_tr, path=path, cat_names=small_cat_vars, cont_names=small_cont_vars).split_none().label_empty().databunch(bs=len(small_df_test_tr.index))
    
xb,yb=databunch.one_batch()[0],databunch.y
cb_handler=CallbackHandler(learn.callbacks)
means=torch.FloatTensor(list(learn.data.label_list. get_processors()[0][0].procs[2].means.values()))
stds =torch.FloatTensor(list(learn.data.label_list. get_processors()[0][0].procs[2].stds.values()))
model=learn.model.eval()
history_df=pd.DataFrame(index=small_df_test_tr['Date'].apply(pd.to_datetime))
out = model(*xb)
history_df['Realised']=small_df_test_tr['Revenue'].values
#history_df['Predicted']=exp(out.detach())
history_df['Predicted']=out.detach()
history_df['Predicted']=history_df['Predicted'].apply(lambda x: x[0].item())

Preparing data and creating base case prediction

In [27]:
small_cat_vars=['Product Category (Enhanced E-commerce)','Dayofweek', 'Year', 'Month', 'Day']
print (small_df_test.mean(axis=0))
#small_df_test_tr=small_df_test.copy()
small_df_test_tr=spend_with_sales1[small_cat_vars + small_cont_vars + ['Revenue','Date']]
small_df_test_tr=small_df_test_tr[small_df_test_tr['Month']>5]
#print(data.label_list.train.x.classes)
for proc in data.label_list. get_processors()[0][0].procs:
    #if hasattr(proc,"categories"):
        #print (proc.categories)

    proc.apply_test(small_df_test_tr)

databunch=TabularList.from_df(small_df_test_tr, path=path, cat_names=small_cat_vars, cont_names=small_cont_vars).split_none().label_empty().databunch(bs=len(small_df_test_tr.index))

xb,yb=databunch.one_batch()[0],databunch.y
#print(dir(xb))
print (small_df_test_tr.shape,len(xb[0]))
cb_handler=CallbackHandler(learn.callbacks)
means=torch.FloatTensor(list(learn.data.label_list. get_processors()[0][0].procs[2].means.values()))
stds =torch.FloatTensor(list(learn.data.label_list. get_processors()[0][0].procs[2].stds.values()))
model=learn.model.eval()
#print (learn.data.valid_ds.conts)
#print (learn.data.one_batch(ds_type=DatasetType.Test)[0][1].mean(axis=0))
#print (stds, means)
#print(learn.data.one_batch(ds_type=DatasetType.Train)[0][1].mean(axis=0))
lr=10
num_iter=0


current_sales=0

output_df=pd.DataFrame(index=small_df_test_tr['Date'].apply(pd.to_datetime))
out = model(*xb)
output_df['Original']=out.detach()
currect_dist=(xb[1]*stds.T+means.T).mean(axis=0)

Dayofweek        2.98243
Year                2017
Month                 11
Day              15.5308
Date                 NaT
Offline Spend    3558.88
Online Spend     2355.85
dtype: object
(2278, 9) 2278


In [33]:
class FakeOptimizer():
    def step(self): pass
    def zero_grad(self): pass

def find_optimal_distribution(total_spend_increase):
    multiplier=torch.tensor([1.0+total_spend_increase,1.0+total_spend_increase])
    xb_grid=xb.copy()
    xb_grid[1] = xb_grid[1]*multiplier
    max_sales=0
    for x in range(-10,10):
        out = model(*xb_grid)
        print ("Average spend processed "+ str((xb_grid[1]*stds.T+means.T).mean(axis=0))+"__"+str((xb_grid[1]*stds.T+means.T).mean(axis=0).mean()))
        #current_sales=exp(out.detach()).mean(axis=0)
        #todo:check exponent
        current_sales=out.detach().mean(axis=0)
        print("Func "+str(current_sales))
        if current_sales>max_sales:
            print("Replacing "+str(max_sales))
            max_sales=current_sales
            #output_df['Optimised']=exp(out.detach())
            output_df['Optimised']=out.detach()
            new_dist=(xb_grid[1]*stds.T+means.T).mean(axis=0)
        direction=torch.tensor([x*180.0,-x*180.0])
        direction= direction-direction.mean()
        direction= direction/stds.T

        print(direction)
        #grad_norm=grad*stds
        xb_grid[1] = xb[1]+direction
    return (new_dist,output_df)

In [34]:
w= interactive(find_optimal_distribution, total_spend_increase=(-0.5, 0.5))
display(w)
(new_dist,output_df)=w.result

interactive(children=(FloatSlider(value=0.0, description='total_spend_increase', max=0.5, min=-0.5), Output())…

In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = ["Offline", "Online"]

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=currect_dist, name="Existing allocation"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=new_dist, name="Optimized allocation"),
              1, 2)
fig.update_layout(title_text="Redistrbuting the marketing budgets...")

# references
A time delay neural network architecture for efficient modeling of long
temporal contexts https://www.danielpovey.com/files/2015_interspeech_multisplice.pdf
Neural Interaction Detection http://www.interpretable-ml.org/nips2017workshop/papers/09.pdf
detecting statistical interactions from neural network weights https://openreview.net/pdf?id=ByOfBggRZ
https://www.fast.ai/2018/04/29/categorical-embeddings/
